# Adquisición y preprocesamiento

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

files = os.listdir('d\\')

list_of_dfs = []
for file in files:
    with open(os.path.join('d\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
        
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        allData = allData.merge(aux, left_index=True, right_index=True)
        
        list_of_dfs.append(allData)
data = pd.concat(list_of_dfs, ignore_index=True)

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

data
# En data tenemos la información de todos los sujetos

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,F4,F8,AF4,Fp2,Fz,Cz,valence,arousal,dominance,liking
0,0.948232,10.260175,1.013050,-7.658428,-1.811108,11.011411,3.026008,-2.380048,3.978952,-9.657708,...,4.239575,4.888393,0.596471,0.589618,-2.276449,-0.109300,7.71,7.60,6.90,7.83
1,1.653335,12.795443,-1.067832,-3.267558,-4.783876,7.402976,2.676232,-3.614201,-1.434440,-0.906298,...,4.557239,6.007259,-1.881391,-4.831903,-1.739787,-6.518661,7.71,7.60,6.90,7.83
2,3.013726,10.426192,3.908249,0.701542,-0.522649,1.120469,2.046996,-4.286566,-6.174767,9.408599,...,0.636801,2.921478,-4.484906,-8.186728,0.555901,-11.727187,7.71,7.60,6.90,7.83
3,1.495061,8.229207,6.094405,2.959722,1.299854,-0.832024,2.192056,-5.170547,-6.571542,14.101073,...,-2.965047,-0.679860,-4.483488,-7.905437,-1.168129,-9.051847,7.71,7.60,6.90,7.83
4,-1.264836,3.751782,4.145906,3.459897,-0.916779,-0.784404,-4.694002,-5.332026,-7.793136,16.284187,...,3.248115,-1.103246,-2.547386,-4.419073,-1.319219,-4.072682,7.71,7.60,6.90,7.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322555,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.214036,-0.050429,...,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,5.10,7.12,6.17,5.97
322556,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.215036,-0.050429,...,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,5.10,7.12,6.17,5.97
322557,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.215036,-0.050429,...,-0.101962,0.036677,-0.007930,0.081379,-0.082538,-0.009341,5.10,7.12,6.17,5.97
322558,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.214036,-0.050429,...,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,5.10,7.12,6.17,5.97


# Extracción de características

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Lectura mediante pandas
dictRaw = pd.read_pickle('d\\s01.dat')

labels = dictRaw.get('labels')

dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

data = dictRaw.get('data')

df_videos = {}
#Recorrido de cada una de las columnas de la tabla 3d, y guardado en una una lista de dataframes para cada uno de los videos.
for i in range(40):
    df_videos[i] = pd.DataFrame(data[i])
    df_videos[i] = df_videos[i].transpose()
    df_videos[i].columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature']
    df_videos[i].drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)


In [22]:
print(df_videos)

{0:            FP1       AF3        F7        F3       FC1       FC5        T7  \
0     0.948232  0.124707 -2.216511  1.005734  5.095919  1.206457  3.866461   
1     1.653335  1.390083  2.292017  1.297927  5.000869  0.599485  3.280028   
2     3.013726  1.835099  2.746369  2.367689  4.176166  0.869299  0.656367   
3     1.495061 -1.110738 -2.364459 -0.231850  0.354056  1.974166 -3.077413   
4    -1.264836 -2.590622 -2.309528 -1.662444 -4.154098 -0.091352 -2.857734   
...        ...       ...       ...       ...       ...       ...       ...   
8059  5.582611  5.416049  6.221588  4.856169  0.298991  1.976263  2.746697   
8060  2.171023  4.085006  4.290778 -0.038786 -0.268494  0.622648  2.022941   
8061 -2.826489 -2.987021 -2.637078 -4.582649 -0.879462 -2.117416 -3.322173   
8062 -4.477230 -6.287809 -7.406510 -4.403024 -2.608805 -3.706191 -4.181635   
8063 -3.676928 -4.474290 -6.755904 -1.373118 -2.170391 -3.781734 -1.693248   

            C3       CP1       CP5  ...        C4        T8

In [23]:
    # Frecuencia de muestreo
    fs = 128
    # Window
    window = "hann"
    # Length of each segment
    # nperseg = 256 por defecto
    # noverlap
    # Por defecto a None, if None : noverlap = nperseg / 2
    # [...]

    # Definicion de bandas
    eeg_bands = {'Delta': (1, 4),
                'Theta': (4, 8),
                'Alpha': (8, 14),
                'Beta': (14, 31),
                'Gamma': (31, 50)}



In [24]:
# FOURIER
import scipy
from scipy import signal

def do_stft(channel):
    # Array 
    x = df_videos[0][channel]

    # Array of the sample frequency, Array of the segment times, STFT of x
    f, t , Zxx = scipy.signal.stft(x, fs, window)

    #print(np.abs(Zxx))
    '''
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    '''
    return Zxx


In [25]:
# Array of the sample frequency, Array of the segment times, Spectogram of x
'''f, t , Sxx = scipy.signal.spectrogram(x, fs, window)

plt.pcolormesh(t, f, np.abs(Sxx), vmin=0, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()'''

"f, t , Sxx = scipy.signal.spectrogram(x, fs, window)\n\nplt.pcolormesh(t, f, np.abs(Sxx), vmin=0, shading='gouraud')\nplt.ylabel('Frequency [Hz]')\nplt.xlabel('Time [sec]')\nplt.show()"

In [26]:
#np.abs(Sxx)

In [27]:
#SEPARACION EN LAS DISTINTAS BANDAS DE FRECUENCIA

def make_bands(Zxx):
    #Obtener valores reales de STFT, solo positivos.
    values = np.absolute(Zxx)

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    #print(fft_freq)

    # Obtención de cada una de las bandas de frecuencia
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = values[freq_ix]

    #print(eeg_band_fft)
    '''
    eeg_band_fft_means = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft_means[band] = np.mean(values[freq_ix])

    #print(eeg_band_fft_means)

    #Graficación de las medias de los datos
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft_means[band] for band in eeg_bands]
    ax = df.plot.bar(x='band', y='val', legend=False)
    ax.set_xlabel("EEG band")
    ax.set_ylabel("Mean band Amplitude")
    '''
    #print(df)

    #print("\n\n")

    '''
    print("Shape de Zxx: ", Zxx.shape)
    print("Shape de Delta: ", eeg_band_fft['Delta'].shape)
    print("Shape de Theta: ", eeg_band_fft['Theta'].shape)
    print("Shape de Alpha: ", eeg_band_fft['Alpha'].shape)
    print("Shape de Beta: ", eeg_band_fft['Beta'].shape)
    print("Shape de Gamma: ", eeg_band_fft['Gamma'].shape)
    '''
    return eeg_band_fft


In [28]:
# Ahora hay que hacer las medias sobre cada banda de frecuencia. La media entre cada una de las posiciones de los bloques. Hacer la media entre todas las posiciones 0, 1, 2... 63 de cada uno de los bloques
import numpy as np

def channel_freq(eeg_band_fft):
        freq = np.zeros((5, 64))
        # Se va a recorrer {Delta, Theta, Alpha, Beta, Gamma}
        for band in eeg_bands:
                # Para cada una de las 64 posiciones del array
                for j in range (0, 64):
                        val = []
                        # Se va a recorrer cada uno de los arrays que hay en cada una de las bandas
                        for i in range (0, len(eeg_band_fft[band])):
                                val.append(eeg_band_fft[band][i][j])
                                # Ejemplo : eeg_band_fft['Delta'][0][0*0 + 0]
                        if (band == "Delta"):
                                freq[0][j] = np.mean(val)
                        elif (band == "Theta"):
                                freq[1][j] = np.mean(val)
                        elif (band == "Alpha"):
                                freq[2][j] = np.mean(val)
                        elif (band == "Beta"):
                                freq[3][j] = np.mean(val)
                        elif (band == "Gamma"):
                                freq[4][j] = np.mean(val)
        return(freq)     # En freq tenemos las frecuencias medias de cada una de las bandas

# Recordamos : Estamos tratando los datos relativos a todos los vídeos de un canal concreto de un sujeto concreto

In [36]:
# Obtener la división por bandas de frecuencia de cada uno de los canales

delta_bands = []
theta_bands = []
beta_bands = []
alpha_bands = []
gamma_bands = []

for channel in ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']:
    Zxx = do_stft(channel)
    eeg_band_fft = make_bands(Zxx)
    print(channel)
    freq = channel_freq(eeg_band_fft)

    delta_bands.append(freq[0])
    theta_bands.append(freq[1])
    alpha_bands.append(freq[2])
    beta_bands.append(freq[3])
    gamma_bands.append(freq[4])
    
    print(freq)
    print("\n\n")
# En freq tenemos la media de todas las bandas de frecuencia de cada uno de los canales

FP1
[[3.12596057e-02 1.41828110e-02 1.34811176e-02 1.37857864e-02
  1.46974597e-02 1.44395256e-02 1.42529609e-02 1.26311306e-02
  1.39736900e-02 1.45192236e-02 1.42657319e-02 1.39324699e-02
  1.32219257e-02 1.42400830e-02 1.38086179e-02 1.39291384e-02
  1.44567323e-02 1.37313508e-02 1.31890758e-02 1.44227219e-02
  1.38455032e-02 1.37186602e-02 1.43189005e-02 1.33956446e-02
  1.41320563e-02 1.46217288e-02 1.33627450e-02 1.38572268e-02
  1.45858871e-02 1.42165579e-02 1.38115880e-02 1.37900705e-02
  1.46285532e-02 1.42530183e-02 1.36834942e-02 1.38216631e-02
  1.38228714e-02 1.45563188e-02 1.33434647e-02 1.34942128e-02
  1.46636150e-02 1.35134187e-02 1.35592723e-02 1.46295938e-02
  1.41744579e-02 1.30466239e-02 1.38069761e-02 1.48043091e-02
  1.43735785e-02 1.31852941e-02 1.36644558e-02 1.45683539e-02
  1.38298711e-02 1.40429521e-02 1.36923492e-02 1.36588841e-02
  1.47285772e-02 1.41581364e-02 1.40963583e-02 1.38118458e-02
  1.34565662e-02 1.43152661e-02 1.41026731e-02 8.82465335e-03]
 [2

In [44]:
# Banda delta
df = pd.DataFrame(delta_bands)
delta_df = df.transpose()
delta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
delta_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.031260,0.052561,0.071918,0.024139,0.018881,0.041480,0.037475,0.104569,0.063523,0.022136,...,0.036610,0.020287,0.074248,0.029771,0.049087,0.029030,0.014418,0.029343,0.065366,0.094424
1,0.014183,0.007454,0.001430,0.028454,0.014384,0.007680,0.000180,0.033965,0.005065,0.021056,...,0.000034,0.008864,0.011633,0.006702,0.017831,0.017972,0.011868,0.002690,0.005342,0.006459
2,0.013481,0.006756,0.000966,0.027919,0.014070,0.007814,0.000701,0.034722,0.006024,0.020203,...,0.000259,0.008642,0.011917,0.007078,0.017887,0.017594,0.011954,0.002937,0.005246,0.006620
3,0.013786,0.006957,0.000825,0.028222,0.014300,0.007839,0.000845,0.035385,0.006238,0.019895,...,0.000481,0.008819,0.012103,0.007400,0.018093,0.017689,0.012037,0.003470,0.005436,0.006613
4,0.014697,0.008138,0.002044,0.029654,0.014677,0.007798,0.000285,0.035284,0.005482,0.020913,...,0.000241,0.009115,0.012272,0.007413,0.018305,0.018390,0.012587,0.003113,0.005882,0.006444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.013812,0.007707,0.002081,0.028439,0.013945,0.007642,0.000683,0.035172,0.005693,0.020464,...,0.000057,0.008813,0.012317,0.008434,0.018468,0.017910,0.012436,0.003108,0.005388,0.007006
60,0.013457,0.006827,0.000638,0.027736,0.014192,0.007759,0.000363,0.034697,0.005858,0.020023,...,0.000614,0.008717,0.011990,0.007227,0.017928,0.017700,0.011580,0.002911,0.004903,0.007056
61,0.014315,0.007379,0.000768,0.028483,0.014212,0.008106,0.000460,0.034698,0.005556,0.020679,...,0.000410,0.008940,0.012026,0.006686,0.018043,0.018135,0.012332,0.002791,0.005641,0.006333
62,0.014103,0.007223,0.001178,0.028334,0.014221,0.007770,0.000341,0.034777,0.005512,0.020838,...,0.000012,0.008853,0.011764,0.007270,0.018275,0.017698,0.012233,0.003210,0.005669,0.006505


In [47]:
# Banda theta
df = pd.DataFrame(theta_bands)
theta_df = df.transpose()
theta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
theta_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.028524,0.051137,0.075335,0.020816,0.026039,0.047910,0.038999,0.123136,0.067368,0.029551,...,0.037697,0.024038,0.082446,0.035713,0.060313,0.039399,0.019923,0.030543,0.064529,0.099619
1,0.000282,0.000220,0.000504,0.000263,0.000301,0.000416,0.000656,0.001083,0.000723,0.000657,...,0.000340,0.000117,0.000485,0.000730,0.000333,0.000085,0.000368,0.000384,0.000138,0.000236
2,0.000581,0.000650,0.000290,0.000639,0.000289,0.000292,0.000450,0.000467,0.000451,0.000353,...,0.000288,0.000227,0.000162,0.000219,0.000232,0.000422,0.000395,0.000339,0.000481,0.000289
3,0.000227,0.000455,0.000403,0.000363,0.000162,0.000181,0.000607,0.000688,0.000691,0.000733,...,0.000211,0.000148,0.000248,0.000343,0.000249,0.000435,0.000360,0.000656,0.000327,0.000227
4,0.000852,0.001015,0.001158,0.001572,0.000618,0.000362,0.000228,0.000528,0.000311,0.000497,...,0.000336,0.000508,0.000441,0.000236,0.000358,0.000622,0.000533,0.000598,0.000717,0.000582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.000183,0.000545,0.001251,0.000206,0.000266,0.000452,0.000391,0.000400,0.000094,0.000159,...,0.000297,0.000255,0.000383,0.001498,0.000565,0.000327,0.000388,0.000247,0.000019,0.000365
60,0.000619,0.000584,0.000708,0.000769,0.000169,0.000307,0.000177,0.000253,0.000412,0.000610,...,0.000427,0.000078,0.000106,0.000351,0.000136,0.000324,0.000681,0.000087,0.000621,0.000480
61,0.000497,0.000573,0.000705,0.000263,0.000211,0.000291,0.000111,0.000300,0.000369,0.000413,...,0.000634,0.000268,0.000100,0.000761,0.000212,0.000280,0.000559,0.000257,0.000488,0.000492
62,0.000180,0.000103,0.000255,0.000232,0.000139,0.000239,0.000072,0.000253,0.000240,0.000414,...,0.000370,0.000061,0.000331,0.000113,0.000281,0.000273,0.000158,0.000428,0.000305,0.000222


In [48]:
#Bnada alpha
df = pd.DataFrame(alpha_bands)
alpha_df = df.transpose()
alpha_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
alpha_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.037022,0.057384,0.084875,0.023901,0.033916,0.056246,0.045340,0.132978,0.072455,0.032813,...,0.039509,0.023606,0.090150,0.043854,0.069744,0.048916,0.031195,0.037202,0.068684,0.104461
1,0.000460,0.000276,0.000962,0.000655,0.000641,0.000882,0.001260,0.001961,0.001239,0.001096,...,0.000415,0.000360,0.000888,0.001442,0.000736,0.000085,0.000846,0.000846,0.000515,0.000251
2,0.000978,0.001217,0.000799,0.001404,0.000721,0.000883,0.000965,0.001241,0.000855,0.000532,...,0.000884,0.000561,0.000376,0.000499,0.000528,0.000892,0.001035,0.001084,0.001309,0.000847
3,0.000466,0.001031,0.000978,0.001188,0.000328,0.000603,0.001123,0.001443,0.001228,0.001251,...,0.000216,0.000556,0.000822,0.000931,0.000705,0.001187,0.001053,0.001357,0.000992,0.000738
4,0.001351,0.001685,0.001985,0.002769,0.001122,0.001002,0.000585,0.000932,0.000643,0.000863,...,0.001021,0.001214,0.001103,0.000441,0.000683,0.001206,0.000870,0.001640,0.001559,0.001503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.000255,0.001091,0.002334,0.000622,0.000423,0.000977,0.000668,0.000761,0.000262,0.000362,...,0.000570,0.000672,0.000741,0.002751,0.001161,0.000857,0.000811,0.000663,0.000126,0.000567
60,0.001046,0.001147,0.001545,0.001452,0.000535,0.000654,0.000452,0.000466,0.000995,0.001137,...,0.000868,0.000194,0.000202,0.000890,0.000251,0.000736,0.001228,0.000209,0.001106,0.000896
61,0.000964,0.001485,0.001722,0.000547,0.000640,0.000695,0.000169,0.000758,0.000967,0.001022,...,0.001720,0.000693,0.000222,0.001478,0.000521,0.000396,0.001374,0.000651,0.001175,0.000984
62,0.000331,0.000222,0.000659,0.000691,0.000455,0.000512,0.000191,0.000843,0.000531,0.000779,...,0.000780,0.000096,0.000697,0.000387,0.000520,0.000619,0.000330,0.000989,0.000453,0.000323


In [49]:
# Banda beta
df = pd.DataFrame(beta_bands)
beta_df = df.transpose()
beta_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
beta_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.118747,0.092576,0.167347,0.235697,0.132488,0.129899,0.230018,0.387798,0.241469,0.131303,...,0.083550,0.116688,0.170983,0.210672,0.208014,0.107983,0.238489,0.226860,0.231235,0.187659
1,0.125937,0.089597,0.305336,0.299316,0.160961,0.174106,0.245749,0.297463,0.197775,0.020053,...,0.153902,0.117805,0.129155,0.172834,0.177079,0.160124,0.246274,0.301997,0.277116,0.252569
2,0.160244,0.215023,0.206206,0.421048,0.083055,0.164993,0.048812,0.275211,0.080018,0.177514,...,0.204638,0.173886,0.107606,0.038059,0.076446,0.193331,0.261205,0.133968,0.311478,0.238571
3,0.204361,0.256108,0.277600,0.378988,0.146304,0.304903,0.032897,0.223238,0.130313,0.109111,...,0.353223,0.191633,0.208356,0.083504,0.125587,0.160259,0.220168,0.400429,0.340836,0.340297
4,0.080042,0.140640,0.154110,0.118205,0.121354,0.237176,0.180139,0.216972,0.221492,0.177714,...,0.201636,0.171767,0.270650,0.308517,0.183990,0.163476,0.146289,0.217210,0.092168,0.147962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.133576,0.253943,0.254969,0.134775,0.103440,0.159173,0.095577,0.124083,0.090134,0.109191,...,0.085561,0.108518,0.066298,0.132452,0.085148,0.097314,0.232223,0.064645,0.185322,0.172662
60,0.030642,0.200201,0.334437,0.173985,0.110353,0.115572,0.078218,0.110039,0.030065,0.084798,...,0.158477,0.078067,0.077241,0.279271,0.115306,0.082853,0.197771,0.116800,0.161707,0.125799
61,0.030728,0.085722,0.180156,0.179978,0.199230,0.052653,0.065464,0.116711,0.202665,0.145379,...,0.180148,0.066674,0.102518,0.269354,0.058490,0.129473,0.128674,0.029818,0.098615,0.047634
62,0.121688,0.193025,0.169838,0.151311,0.134733,0.108242,0.077591,0.207264,0.247556,0.167156,...,0.233609,0.097809,0.035670,0.123733,0.059134,0.088625,0.131303,0.183220,0.153181,0.140757


In [50]:
# Banda gamma
df = pd.DataFrame(gamma_bands)
gamma_df = df.transpose()
gamma_df.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
gamma_df

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,C4,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz
0,0.278582,0.346590,0.435905,0.405591,0.290418,0.170205,0.341940,0.267419,0.265129,0.214697,...,0.245235,0.343467,0.278941,0.272668,0.348843,0.343440,0.544720,0.526994,0.626883,0.491158
1,0.199790,0.338136,0.438542,0.416401,0.180684,0.276907,0.280066,0.459100,0.215868,0.202358,...,0.366517,0.239432,0.397050,0.516301,0.309006,0.210038,0.464953,0.490404,0.451991,0.478594
2,0.431083,0.544522,0.459211,0.314146,0.170494,0.522687,0.710474,0.491338,0.643143,0.712175,...,0.696379,0.475109,0.374063,0.704076,0.389919,0.282175,0.301442,0.582425,0.440806,0.581749
3,0.686639,0.861247,0.954107,0.656273,0.560976,0.266057,0.309216,0.813007,0.368318,0.488949,...,0.605139,0.294303,0.262043,0.573816,0.442543,0.417170,0.923477,0.506173,0.917567,0.658440
4,0.431288,0.421440,0.574657,0.620030,0.237001,0.567875,0.492946,0.680043,0.557123,0.426919,...,0.621795,0.491540,0.394812,0.746486,0.378508,0.302318,0.453386,0.487880,0.446476,0.483455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.583295,0.581292,0.593846,0.625438,0.363983,0.391033,0.348365,0.453730,0.409404,0.289958,...,0.393667,0.309062,0.223661,0.289441,0.166583,0.382282,0.499160,0.383242,0.546279,0.532626
60,0.429290,0.563609,0.538092,0.538054,0.485530,0.460695,0.237030,0.437027,0.318135,0.315305,...,0.674856,0.436052,0.270789,0.733443,0.329807,0.344647,0.419154,0.255771,0.460052,0.477843
61,0.391829,0.324418,0.378128,0.515608,0.403551,0.280977,0.196055,0.585757,0.363345,0.323040,...,0.415296,0.376019,0.353103,0.463374,0.299269,0.280332,0.270452,0.242589,0.292435,0.354162
62,0.469988,0.502568,0.503354,0.481382,0.301898,0.413436,0.363403,0.435773,0.272694,0.308230,...,0.300602,0.279362,0.284202,0.384229,0.418954,0.434148,0.525435,0.420806,0.461438,0.495458


# En este punto lo que hemos hecho ha sido reducir de una matriz (40 videos x 40 canales x 8064) datos que teníamos al principio a (40 videos x 32 canales x 64 datos) para cada una de las bandas